In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from functools import reduce
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

In [17]:
conn = sq.connect('../../../../Data-Pipelines/Outputs/Dem_Transpo_Housing_Collection.db')
sql_query = pd.read_sql('SELECT * FROM [ALICE_County]', conn)
data = pd.DataFrame(sql_query)
thelist = ['Cheatham County, Tennessee', 'Davidson County, Tennessee', 'Dickson County, Tennessee', 'Houston County, Tennessee', 'Humphreys County, Tennessee', 
           'Montgomery County, Tennessee', 'Robertson County, Tennessee', 'Rutherford County, Tennessee', 
          'Stewart County, Tennessee', 'Sumner County, Tennessee', 'Trousdale County, Tennessee', 'Williamson County, Tennessee', 'Wilson County, Tennessee']
data = data.loc[(data['GEO.display_label'].isin(thelist))].reset_index(drop = True)
years = [2019, 2021]
data = data.loc[(data['Year'].isin(years))].reset_index(drop = True)

In [18]:
data.head()

,Year,GEO.id2,GEO.display_label,County,State,State Abbr,Households,Poverty Households,ALICE Households,Above ALICE Households,ALICE Threshold - HH under 65,ALICE Threshold - HH 65 years and over,Source: American Community Survey
0,2019,47021,"Cheatham County, Tennessee",Cheatham,Tennessee,TN,15089,1437,5809,7843,60000,60000,5-Year
1,2021,47021,"Cheatham County, Tennessee",Cheatham,Tennessee,TN,15366,1503,4801,9062,60000,60000,5-Year
2,2019,47037,"Davidson County, Tennessee",Davidson,Tennessee,TN,289427,32169,102562,154696,60000,60000,1-Year
3,2021,47037,"Davidson County, Tennessee",Davidson,Tennessee,TN,316273,42726,100267,173280,60000,60000,1-Year
4,2019,47043,"Dickson County, Tennessee",Dickson,Tennessee,TN,19198,2438,8112,8648,60000,50000,5-Year


In [19]:
data = data[['County', 'Year', 'Households', 'Poverty Households', 'ALICE Households']]

In [20]:
data['Year'] = data['Year'].astype(str)

In [21]:
data = data.set_index(['County', 'Year']).transpose()

In [22]:
data.head()

County             Cheatham        Davidson         Dickson        Houston       Humphreys       Montgomery        Robertson        Rutherford  \
Year                   2019   2021     2019    2021    2019   2021    2019  2021      2019  2021       2019   2021      2019   2021       2019   
Households            15089  15366   289427  316273   19198  19663    2878  2860      6763  6688      76974  84145     26577  26536     113797   
Poverty Households     1437   1503    32169   42726    2438   2148     552   485      1148   746       9300   8773      2490   3125      11451   
ALICE Households       5809   4801   102562  100267    8112   6885    1015  1040      2229  1987      29576  28788      8959   8773      49453   

County                     Stewart       Sumner        Trousdale       Williamson        Wilson         
Year                  2021    2019  2021   2019   2021      2019  2021       2019   2021   2019   2021  
Households          121944    5178  5091  70098  77760      3189  3349      84393  85311  51618  55047  
Poverty Households    9994     706   705   5159   6538       276   407       4076   3879   3815   4890  
ALICE Households     45904    1635  1422  23720  26467      1637  1333      22249  21992  18836  18870

In [24]:
geos = ['Cheatham', 'Davidson', 'Dickson', 'Houston', 'Humphreys', 'Montgomery', 'Robertson', 'Rutherford', 
          'Stewart', 'Sumner', 'Trousdale', 'Williamson', 'Wilson']
years = ['2019', '2021']

In [25]:
for year in years:
    thelist = [data[('Cheatham', '{}'.format(year))], data[('Davidson', '{}'.format(year))] + data[('Dickson', '{}'.format(year))] + 
              + data[('Houston', '{}'.format(year))]+ data[('Humphreys', '{}'.format(year))] + data[('Montgomery', '{}'.format(year))] + 
              data[('Robertson', '{}'.format(year))] + data[('Rutherford', '{}'.format(year))] + data[('Stewart', '{}'.format(year))] + 
              data[('Sumner', '{}'.format(year))]+ data[('Trousdale', '{}'.format(year))] + data[('Williamson', '{}'.format(year))] + 
              data[('Wilson', '{}'.format(year))]]
    data[('GNRC', '{}'.format(year))] = sum(thelist)
    

In [26]:
data.head()

County             Cheatham        Davidson         Dickson        Houston       Humphreys       Montgomery        Robertson        Rutherford  \
Year                   2019   2021     2019    2021    2019   2021    2019  2021      2019  2021       2019   2021      2019   2021       2019   
Households            15089  15366   289427  316273   19198  19663    2878  2860      6763  6688      76974  84145     26577  26536     113797   
Poverty Households     1437   1503    32169   42726    2438   2148     552   485      1148   746       9300   8773      2490   3125      11451   
ALICE Households       5809   4801   102562  100267    8112   6885    1015  1040      2229  1987      29576  28788      8959   8773      49453   

County                     Stewart       Sumner        Trousdale       Williamson        Wilson           GNRC          
Year                  2021    2019  2021   2019   2021      2019  2021       2019   2021   2019   2021    2019    2021  
Households          121944    5178  5091  70098  77760      3189  3349      84393  85311  51618  55047  765179  820033  
Poverty Households    9994     706   705   5159   6538       276   407       4076   3879   3815   4890   75017   85919  
ALICE Households     45904    1635  1422  23720  26467      1637  1333      22249  21992  18836  18870  275792  268529

In [27]:
data = data.transpose().reset_index(drop = False)

In [28]:
data.head()

,County,Year,Households,Poverty Households,ALICE Households
0,Cheatham,2019,15089,1437,5809
1,Cheatham,2021,15366,1503,4801
2,Davidson,2019,289427,32169,102562
3,Davidson,2021,316273,42726,100267
4,Dickson,2019,19198,2438,8112


In [30]:
data['% Households in Poverty'] = data['Poverty Households']/data['Households']
data['% Households ALICE'] = data['ALICE Households']/data['Households']

In [31]:
data.head()

,County,Year,Households,Poverty Households,ALICE Households,% Households in Poverty,% Households ALICE
0,Cheatham,2019,15089,1437,5809,0.095235,0.384982
1,Cheatham,2021,15366,1503,4801,0.097813,0.312443
2,Davidson,2019,289427,32169,102562,0.111147,0.354362
3,Davidson,2021,316273,42726,100267,0.135092,0.317027
4,Dickson,2019,19198,2438,8112,0.126992,0.422544


In [32]:
data.to_csv('../data/alice.csv')